이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보는 실습을 진행코자 한다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를 만한 Movielens 데이터이다.

유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기별로 있다. MovieLens 1M Dataset 사용을 권장하며, 별점 데이터는 대표적인 explicit 데이터이다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있다. 별점을 시청 횟수로 해석해서 생각한다.

또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다는 가정을 세웠다.

Cloud Storage에 미리 업로드된 ml-1m 폴더 내 파일을 심볼릭 링크로 개인 storage에 연결한다. 이는 Cloud shell에서 아래의 명령어를 입력함으로써 수행 가능하다.

```
$ mkdir -p ~/aiffel/recommendata_iu/data/ml-1m
$ ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-
```

# Step 0. 사전 준비
# (라이브러리 및 데이터셋 불러들이기)

In [311]:
# 프로젝트에 필요한 주요 라이브러리 버전 확인
import numpy as np
import pandas as pd
import scipy
import implicit

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


In [312]:
# 유저들의 영화 평점을 나타낸 데이터셋을 먼저 불러들인다.
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
print(len(ratings))
ratings.head()

1000209


,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# Step 1. 데이터 전처리

In [313]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [314]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [315]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [316]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


여기까지가 기본적인 전처리이며, 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하는 시간을 본격적으로 가지게 된다!!

이에 앞서 문자열에 관련한 전처리를 하고 관심 있는 컬럼과 그렇지 않은 컬럼을 솎아내는 과정도 필요하다.

In [317]:
# 검색을 쉽게 하기 위해 문자열을 소문자로 바꿔주기
movies['title'] = movies['title'].str.lower()
movies['genre'] = movies['genre'].str.lower()

# genre 전처리 추가하기
movies.head(10)

,movie_id,title,genre
0,1,toy story (1995),animation|children's|comedy
1,2,jumanji (1995),adventure|children's|fantasy
2,3,grumpier old men (1995),comedy|romance
3,4,waiting to exhale (1995),comedy|drama
4,5,father of the bride part ii (1995),comedy
5,6,heat (1995),action|crime|thriller
6,7,sabrina (1995),comedy|romance
7,8,tom and huck (1995),adventure|children's
8,9,sudden death (1995),action
9,10,goldeneye (1995),action|adventure|thriller


In [318]:
# 영화 제목이 포함된 메타 데이터와 영화 평가 데이터를 합체!!
df = pd.merge(movies, ratings, on='movie_id')
df

,movie_id,title,genre,user_id,counts,timestamp
0,1,toy story (1995),animation|children's|comedy,1,5,978824268
1,1,toy story (1995),animation|children's|comedy,6,4,978237008
2,1,toy story (1995),animation|children's|comedy,8,4,978233496
3,1,toy story (1995),animation|children's|comedy,9,5,978225952
4,1,toy story (1995),animation|children's|comedy,10,5,978226474
...,...,...,...,...,...,...
836473,3952,"contender, the (2000)",drama|thriller,5682,3,1029457829
836474,3952,"contender, the (2000)",drama|thriller,5812,4,992072099
836475,3952,"contender, the (2000)",drama|thriller,5831,3,986223125
836476,3952,"contender, the (2000)",drama|thriller,5837,4,1011902656


In [319]:
# 사용할 컬럼만 남겨주는 과정이며, 관심 항목이 아닌 부분은 모두 배제하기
using_cols = ['user_id', 'title', 'genre', 'counts']
df = df[using_cols]
df.head(10)

,user_id,title,genre,counts
0,1,toy story (1995),animation|children's|comedy,5
1,6,toy story (1995),animation|children's|comedy,4
2,8,toy story (1995),animation|children's|comedy,4
3,9,toy story (1995),animation|children's|comedy,5
4,10,toy story (1995),animation|children's|comedy,5
5,18,toy story (1995),animation|children's|comedy,4
6,19,toy story (1995),animation|children's|comedy,5
7,21,toy story (1995),animation|children's|comedy,3
8,23,toy story (1995),animation|children's|comedy,4
9,26,toy story (1995),animation|children's|comedy,3


나는  `user_id`, `title`, `genre`, `counts`의 4개의 컬럼을 남기기로 했다. 그리고 본격적인 분석을 시작해 보기로 했다.

# Step 2. 데이터 분석

1) ratings에 있는 유니크한 사용자 개수  
2) ratings에 있는 유니크한 영화 수  
3) 가장 인기 있는 영화 30개(인기순)  

In [320]:
# 첫번째 유저가 어떤 영화를 관람했는지 확인
condition = (df['user_id']== ratings.loc[0, 'user_id'])
df.loc[condition]

,user_id,title,genre,counts
0,1,toy story (1995),animation|children's|comedy,5
19773,1,pocahontas (1995),animation|children's|musical|romance,5
35123,1,apollo 13 (1995),drama,5
54895,1,star wars: episode iv - a new hope (1977),action|adventure|fantasy|sci-fi,4
116019,1,schindler's list (1993),drama|war,5
119008,1,"secret garden, the (1993)",children's|drama,4
130190,1,aladdin (1992),animation|children's|comedy|musical,4
138999,1,snow white and the seven dwarfs (1937),animation|children's|musical,4
139706,1,beauty and the beast (1991),animation|children's|musical,5
142605,1,fargo (1996),crime|drama|thriller,4


첫 번째 유저는 대체로 50여편의 영화를 관람한 듯 보인다. 장르도 보자면 주로 드라마 장르의 영화를 즐겨 본 듯하다. 평가도 생각보다 후한 편이다.

In [321]:
# (1) unique한 유저 수
df['user_id'].nunique()

6039

In [322]:
# (2) unique한 영화 수
df['title'].nunique()

3628

In [323]:
# (3) 인기 많은 영화 Top 30
movie_count = df.groupby('title')['counts'].sum()
movie_count_result = movie_count.sort_values(ascending=False).head(30)
movie_count_result = pd.DataFrame(movie_count_result)
movie_count_result

,counts
title,
american beauty (1999),14449
star wars: episode iv - a new hope (1977),13178
star wars: episode v - the empire strikes back (1980),12648
saving private ryan (1998),11348
star wars: episode vi - return of the jedi (1983),11303
raiders of the lost ark (1981),11179
"silence of the lambs, the (1991)",11096
"matrix, the (1999)",10903
"sixth sense, the (1999)",10703


rating(counts)을 sum한 값임에 유의하자. 즉, counts란에 해당 영화를 본 횟수가 합쳐져 있다는 걸 의미한다.

In [324]:
# (4) 번외 : 인기 많은 영화 장르 Top 10
genre_count = df.groupby('genre')['counts'].count()
genre_count_result = genre_count.sort_values(ascending=False).head(10)
genre_count_result = pd.DataFrame(genre_count_result)
genre_count_result

,counts
genre,
drama,99388
comedy,94264
comedy|drama,36871
comedy|romance,35888
drama|romance,24835
action|thriller,22675
drama|thriller,16133
horror,15260
thriller,14925


드라마가 가장 인기 있었던 장르였던 거 같다.

In [325]:
# 유저별 몇 번의 영화를 관람했는지에 대한 통계
user_count = df.groupby('user_id')['counts'].sum()
user_count.describe()

count    6039.000000
mean      548.273721
std       606.449800
min         3.000000
25%       154.000000
50%       329.000000
75%       704.000000
max      7590.000000
Name: counts, dtype: float64

In [326]:
user_count

user_id
1        222
2        455
3        190
4         85
5        532
        ... 
6036    2620
6037     727
6038      73
6039     469
6040    1117
Name: counts, Length: 6039, dtype: int64

조사 기간 동안 유저별 평균 548번 가량을 영화를 관람했다는 걸 알 수 있다. 최소 3번, 최대 7590번으로 범위가 꽤 넓다는 것도 알 수 있다.

'star wars: episode v - the empire strikes back (1980)' , 'alien (1979)' ,'sixth sense, the (1999)' ,'terminator 2: judgment day (1991)	' ,'matrix, the (1999)'

# Step 3. 내가 선호하는 영화를 5가지 골라서 rating에 추가하기

이거 하기 전에 사용할 컬럼만 남겨주는 과정을 잠시 거친다. 장르 컬럼은 데이터 분석에 필요해서 임시적으로 남겼을 뿐인데, 이제는 배제를 하도록 한다. 

In [327]:
using_cols = ['user_id', 'title', 'counts']
df = df[using_cols]
df.head(10)

,user_id,title,counts
0,1,toy story (1995),5
1,6,toy story (1995),4
2,8,toy story (1995),4
3,9,toy story (1995),5
4,10,toy story (1995),5
5,18,toy story (1995),4
6,19,toy story (1995),5
7,21,toy story (1995),3
8,23,toy story (1995),4
9,26,toy story (1995),3


In [328]:
# 내가 제일 좋아하는 영화 고르기. 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞출 것!!
my_favorite = ['star wars: episode v - the empire strikes back (1980)' , 'alien (1979)' ,
               'sixth sense, the (1999)' ,'terminator 2: judgment day (1991)	' ,'matrix, the (1999)']

# 'kgh'이라는 user_id가 위 영화를 시청한 횟수(별점)가 5씩이라고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['kgh']*5, 'title': my_favorite, 'counts':[5]*5})

if not ratings.isin({'user_id':['kgh']})['user_id'].any():  # user_id에 'kgh'이라는 데이터가 없다면
    df = df.append(my_playlist)                             # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

df.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,title,counts
836473,5682,"contender, the (2000)",3
836474,5812,"contender, the (2000)",4
836475,5831,"contender, the (2000)",3
836476,5837,"contender, the (2000)",4
836477,5998,"contender, the (2000)",4
0,kgh,star wars: episode v - the empire strikes back...,5
1,kgh,alien (1979),5
2,kgh,"sixth sense, the (1999)",5
3,kgh,terminator 2: judgment day (1991)\t,5
4,kgh,"matrix, the (1999)",5


In [329]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = df['user_id'].unique()
movie_unique = df['title'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [330]:
df['user_id'].nunique()

6040

In [331]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['kgh'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다. 
print(movie_to_idx['alien (1979)'])  

6039
1098


다음으로 indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드를 구현하는 과정이다. dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고바란다.

user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구할 것이며, 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 

In [332]:
temp_user_data = df['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(df):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    df['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = df['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(df):
    print('title column indexing OK!!')
    df['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

df

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,counts
0,0,0,5
1,1,0,4
2,2,0,4
3,3,0,5
4,4,0,5
...,...,...,...
0,6039,1080,5
1,6039,1098,5
2,6039,2507,5
3,6039,3628,5


data의 user_id와 title 컬럼 내 값들이 모두 정수 인덱스 값으로 잘 변경된 걸 확인했다!! 이걸로 훈련을 위한 전처리는 끝!!

# Step 4. CSR matrix 생성

In [333]:
from scipy.sparse import csr_matrix

num_user = df['user_id'].nunique()
num_movie = df['title'].nunique()

csr_data = csr_matrix((df.counts, (df.user_id, df.title)), shape= (num_user, num_movie))
csr_data

<6040x3629 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# Step 5. als_model = AlternatingLeastSquares 모델 구성

In [334]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분. 학습 내용과는 무관!
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 `__init__` 파라미터를 살펴보면 다음과 같다.

* factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
* regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
* use_gpu : GPU를 사용할 것인지
* iterations : epochs와 같은 의미. 데이터를 몇 번 반복해서 학습할 것인지

여기서 factors와 iterations를 늘릴수록 학습 데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 한다.

In [335]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=300, regularization=0.01,
                                    use_gpu=False, iterations=30, dtype=np.float32)

In [336]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3629x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [337]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

# Step 6. 선호도 파악

내가 선호하는 5가지 영화 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보고자 한다!!

방금 모델 학습이 끝났는데 이 선호도 파악을 하기 위해서는 2가지 사항을 살펴봐야 한다.

* kgh 벡터와 matrix, the(1999)의 벡터를 어떻게 만들고 있는지
* 두 벡터를 곱하면 어떤 값이 나올지

이 부분이 잘 파악이 되지는 않지만 우선 코드를 작성을 해보자.

In [338]:
kgh, matrix = user_to_idx['kgh'], movie_to_idx['matrix, the (1999)']
kgh_vector, matrix_vector = als_model.user_factors[kgh], als_model.item_factors[matrix]

In [339]:
kgh_vector

array([ 0.50979656, -0.24819466, -0.09440548, -0.4390926 , -0.07023549,
        0.43910173,  0.26078492, -0.03533255,  0.09421652, -0.24248517,
        0.02524716,  0.5376377 ,  0.38293332,  0.08545118,  0.08976402,
        0.23837923, -0.39416206, -0.13642073, -0.08296317, -0.03208876,
        0.15114488, -0.0542547 , -0.21109377, -0.06674007, -0.16798452,
       -0.21043922,  0.1012484 , -0.24969925, -0.58643425,  0.12309005,
        0.2673609 , -0.08371894,  0.02648275,  0.10525281, -0.09989154,
       -0.41088256, -0.37869054,  0.54109865, -0.09440168, -0.5201408 ,
        0.37532866, -0.18164073, -0.06807723,  0.14064002, -0.3604981 ,
       -0.10098732,  0.17096837, -0.10781826, -0.39349818, -0.2751137 ,
        0.03520151,  0.3448163 ,  0.20629667, -0.55629784, -0.2501605 ,
       -0.20512451, -0.2968154 , -0.350849  , -0.40514815, -0.31825024,
        0.2859526 , -0.20785499,  0.39230093,  0.24752608,  0.08718432,
        0.37215775, -0.40394017,  0.3218801 , -0.5575305 ,  0.15

In [340]:
matrix_vector

array([ 1.13362893e-02, -1.69355161e-02,  8.53332039e-03, -1.81687940e-02,
        7.22092204e-03,  1.11595923e-02,  1.38940243e-02, -1.49291093e-02,
       -1.20274927e-02,  1.55256374e-03, -2.41965484e-02,  3.52625884e-02,
        8.63313209e-03,  9.89714172e-03,  1.35791898e-02,  1.62304565e-02,
       -1.88352782e-02,  1.10084079e-02,  2.69217454e-02,  1.20488238e-02,
        2.07447503e-02,  1.34352762e-02, -3.92199308e-03,  3.76176927e-03,
        9.64972656e-03, -1.89693455e-05,  6.58297539e-03, -2.93148570e-02,
       -6.10572193e-03,  1.82848591e-02,  5.64852729e-03,  9.69595741e-03,
        1.06187603e-02, -7.65133137e-03,  2.26161350e-02,  1.20599354e-02,
        2.19432171e-02,  2.83728801e-02, -1.57892157e-03, -1.25314826e-02,
        2.11060829e-02,  3.25947814e-03,  2.41360068e-02, -1.76151153e-02,
       -3.78868589e-03, -9.62162390e-03,  1.58003997e-02,  3.19799967e-02,
       -3.65495845e-03, -8.33817478e-03,  1.37974246e-04,  1.84063204e-02,
       -5.14443126e-03, -

In [341]:
# kgh과 matrix를 내적하는 코드
np.dot(kgh_vector, matrix_vector)

0.8396358

1차 : factors=100, iterations=15  => 내적값 : 0.55  
2차 : factors=300, iterations=30  => 내적값 : 0.84  

정도가 나왔다. 이게 1에 가까우면 좋다고 하지만 일단 두 번 정도 실험을 해보니 각각 위와 같이 결과가 나왔다. 일단 0.85에서 만족해야겠지만 조금 더 이 값에 대해 생각해 볼 게 있을 거 같다.  
왜냐하면 이 factors와 iterations을 늘려서 1에 가깝게 나온다고 한들 이 모델이 잘 학습했다고 간주하기 어렵다고 했기 때문이다. 학습 데이터에 대해서 fitting은 되었지만 아직 보지 못한 데이터에 대한 검증 과정이 빠졌다는 것이다.

In [342]:
jumanji = movie_to_idx['jumanji (1995)']
jumanji_vector = als_model.item_factors[jumanji]
np.dot(kgh_vector, jumanji_vector)

-0.051596742

In [343]:
titanic = movie_to_idx['titanic (1997)']
titanic_vector = als_model.item_factors[titanic]
np.dot(kgh_vector, titanic_vector)

0.006563507

저는 장르의 유사성으로 해당 결과를 분석해 보고자 한다. 일단 매트릭스(1999)는 SF 및 액션 장르에 속하는 영화이고, 쥬만지(1995)는 판타지 & 모험, 타이타닉(1997)은 재난 및 로맨스 장르의 영화이다.  

본인이 가장 좋아하는 영화 중 하나로 꼽은 매트릭스는 쥬만지와 비교했을 때 장르적인 면에서 접점이 없어서 그런지 내적값이 마이너스가 나온 걸로 보인다. 그럼 매트릭스와 타이타닉의 경우는? 타이타닉 호 침몰이라는 재난 영화와 액션 장르라... 접점이 보이는가?  

쥬만지와의 내적값과 비교했을 때 이 경우는 양수값이 나왔으니 조금은 장르적인 면에서 유사성이 있고 그래서 추천해 줄 수 있는 정도가 조금은 높은 걸까?  

이걸 알려면 데이터에 대한 이해도가 더 높아야 될 것이고 더 좋은 모델을 만들어야 하지 않을까 싶다. 장르적 유사성으로 접근해보니 선호도에 대한 부분을 정확히 알지 못했고 보다 많은 데이터를 학습해야 될 거 같다고 느꼈다.

# Step 7. 비슷한 영화를 추천

이번에는 내가 좋아하는 영화와 비슷한 영화를 추천받아 보는 코드를 짜 볼 것이다.
AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통해서 비슷한 영화 제목을 찾는다. 처음으로는 조금 전에 선호도를 알아보기 위해 했던 matrix를 통해 찾아보겠다.

In [344]:
favorite_movie = 'matrix, the (1999)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(2325, 1.0000001),
 (569, 0.36362967),
 (439, 0.28738913),
 (3628, 0.26903355),
 (1377, 0.26871294),
 (2385, 0.26753756),
 (1386, 0.26722077),
 (957, 0.26697296),
 (57, 0.2665707),
 (2655, 0.2664428),
 (2505, 0.2660637),
 (2698, 0.26480457),
 (1972, 0.26420212),
 (1191, 0.2639185),
 (2432, 0.2634075)]

(영화의 id, 유사도) 에 대한 정수 시퀀스가 Tuple 형태로 반환된 것을 확인할 수 있다. 영화의 id를 다시 영화의 이름으로 매핑시킨다.

In [345]:
# artist_to_idx를 뒤집어 index로부터 artist 이름을 얻는 dict를 생성한다.
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['matrix, the (1999)',
 'terminator 2: judgment day (1991)',
 'fugitive, the (1993)',
 'terminator 2: judgment day (1991)\t',
 'fifth element, the (1997)',
 "mummy's ghost, the (1944)",
 'second jungle book: mowgli & baloo, the (1997)',
 'surviving picasso (1996)',
 'confessional, the (le confessionnal) (1995)',
 'molly (1999)',
 'gambler, the (a játékos) (1997)',
 'beefcake (1999)',
 'see the sea (regarde la mer) (1997)',
 'female perversions (1996)',
 'red dwarf, the (le nain rouge) (1998)']

터미네이터 2나 도망자(fugitive) 등 매트릭스와 유사한 장르의 액션 장르의 영화가 잘 나열된 것을 확인할 수 있었다!!!

In [346]:
# artist_to_idx를 뒤집어 index로부터 artist 이름을 얻는 dict를 생성한다.
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie_2]

['sixth sense, the (1999)',
 'drive me crazy (1999)',
 'terminator 2: judgment day (1991)\t',
 'crazy in alabama (1999)',
 'drunks (1997)',
 'brother minister: the assassination of malcolm x (1994)',
 'dry cleaning (nettoyage à sec) (1997)',
 'skipped parts (2000)',
 'superstar (1999)',
 'pandora and the flying dutchman (1951)',
 'faithful (1996)',
 'faust (1994)',
 'raise the titanic (1980)',
 'regret to inform (1998)',
 'communion (a.k.a. alice, sweet alice/holy terror) (1977)']

몇 번 더 반복해서 확인해 보기 위해 위에 적었던 코드를 함수로 만들고 확인해 보겠다.

In [347]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [348]:
get_similar_movie('alien (1979)')

['alien (1979)',
 'aliens (1986)',
 'jaws (1975)',
 'terminator, the (1984)',
 "actor's revenge, an (yukinojo henge) (1963)",
 'alien escape (1995)',
 'all the vermeers in new york (1990)',
 'belly (1998)',
 'terminator 2: judgment day (1991)\t',
 'year of the horse (1997)']

In [349]:
get_similar_movie('sixth sense, the (1999)')

['sixth sense, the (1999)',
 'communion (a.k.a. alice, sweet alice/holy terror) (1977)',
 'terminator 2: judgment day (1991)\t',
 'crazy in alabama (1999)',
 'belly (1998)',
 'skipped parts (2000)',
 'brother minister: the assassination of malcolm x (1994)',
 'shattered image (1998)',
 'pandora and the flying dutchman (1951)',
 'dry cleaning (nettoyage à sec) (1997)']

두번째로는 SF물이면서 스릴러물이기도 한 에일리언을 적어놓고 돌려보니 에일리언이라는 단어가 들어가는 영화 및 조스와 같은 약간 스릴러 느낌의 작품도 출력됐음을 확인했다.

세번째로는 미스터리/스릴러물 중 본인이 좋아하는 영화로 꼽았던 식스 센스를 적어놓고 돌려보니 오잉?? 코미디/드라마 물이 주로 등장?
[drive me crazy, crazy in alabama, skipped parts, faust, superstar 등등]  
조금 어안이 벙벙했다. 이 장르의 영화 자체가 데이터셋에 많이 없어서 그런가? 물론 맨 끝에 있는 communion(a.k.a. alice, sweet alice/holy terror)라는 호러물로 유사성이 있는 작품도 나오기는 했다만... 메커니즘을 알다가도 모르겠는 느낌적인 느낌~~

아, 유사 장르인지 아닌지를 확인해 보고 싶다면 여기 들어가서 확인해 보라!!!

IMdb 주소 링크 : https://www.imdb.com/title/tt0076150/

# Step 8. 최애 영화 추천

AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 제가 좋아할 만한 영화를 추천받는다. filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument이다!!

In [350]:
user = user_to_idx['kgh']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(249, 0.5710056),
 (1084, 0.45600802),
 (1094, 0.26985323),
 (1122, 0.24312839),
 (569, 0.19727325),
 (1260, 0.19009432),
 (523, 0.17996514),
 (49, 0.17738184),
 (2154, 0.1258766),
 (1196, 0.12467311),
 (439, 0.1239091),
 (3474, 0.12122565),
 (0, 0.11850036),
 (1152, 0.11717812),
 (1779, 0.111899935),
 (1750, 0.10938067),
 (2439, 0.10896732),
 (1377, 0.10720522),
 (1810, 0.10450633),
 (31, 0.103431284)]

In [351]:
[idx_to_movie[i[0]] for i in movie_recommended]

['star wars: episode iv - a new hope (1977)',
 'aliens (1986)',
 'star wars: episode vi - return of the jedi (1983)',
 'terminator, the (1984)',
 'terminator 2: judgment day (1991)',
 'jaws (1975)',
 'blade runner (1982)',
 'usual suspects, the (1995)',
 'rushmore (1998)',
 'alien³ (1992)',
 'fugitive, the (1993)',
 'x-men (2000)',
 'toy story (1995)',
 'back to the future (1985)',
 'exorcist, the (1973)',
 'breakfast club, the (1985)',
 'run lola run (lola rennt) (1998)',
 'fifth element, the (1997)',
 'saving private ryan (1998)',
 'twelve monkeys (1995)']

star wars와 aliens를 추천해 주고 있다. 터미네이터 및 식스 센스를 비롯해 장르의 유사성이 보이는 작품들로 잘 추천을 해 준 걸로 판단할 수 있다. 모델은 왜 이러한 영화들을 추천해 줬을까? AlternatingLeastSquares 클래스에 구현된 explain 메서드를 사용하면 내가 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있다.

In [352]:
aliens = movie_to_idx['star wars: episode iv - a new hope (1977)']
explain = als_model.explain(user, csr_data, itemid=aliens)

이 method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도를 반환한다(합이 콘텐츠의 점수가 된다). 어떤 아티스트들이 이 추천에 얼마나 기여하고 있는 건지도 확인해 보자.

In [353]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('star wars: episode v - the empire strikes back (1980)',
  0.42780966378928253),
 ('alien (1979)', 0.13962150904888987),
 ('matrix, the (1999)', 0.008321326460593434),
 ('terminator 2: judgment day (1991)\t', -0.0001647727191466056),
 ('sixth sense, the (1999)', -0.010992304910957172)]

스타 워즈 에피소드 5의 기여도가 상당히 높게 나왔다. 약 0.45 정도로 말이다. 역시 나는 SF물을 좋아하긴 한가 보다 ㅎㅎ

# 회고

* 우선 이번에도 일상 생활과 관련한 데이터셋을 주제 삼아 실습을 해보니 상당히 재미있는 느낌을 받을 수 있었다.

* 본인의 경우는 SF 장르와 스릴러물, 재난 영화를 좋아하는 편인데, 실제로 맨 마지막 스텝인 최애 영화 추천을 통해 결과를 확인해 보니 내가 좋아하는 장르의 영화를 맞춘 거 같았다. 내 취향을 컴퓨터가 어느 정도 알고 있다는 느낌이 들어 기분이 묘하고 좋았다!!!

* Step 6에서 선호도를 확인하기 위해 내 이름을 가진 벡터와 최애 영화 이름의 벡터를 내적하는 과정을 거쳤는데, 이 값이 가지는 의미를 아직 꿰뚫어 보지 못한 점이 조금은 아쉬웠다. 이번 Node는 전반적으로 따라하기 쉬운 편이었고 일상 생활과 관련되어 있어서 재미를 느낄 수 있는 것은 사실이지만 해당 개념에서 가지는 내적값이 어떤 의미인지를 파악할 수 있는 능력은 다소 필요해 보였다.

* Step 7에서 비슷한 영화를 추천할 때도 식스 센스는 분명 미스터리 스릴러물일텐데, 이 작품과 유사성을 지닌 작품을 소개하는 코드를 짜놓고 보니 결과가 약간 코미디/드라마 계열의 영화가 나왔다는 것이 좀 웃기면서도 이상하다는 생각이 들었다. 메커니즘이 알다가도 모르겠는게 참 신기했다.
